# 세션 이탈
### 사용자의 첫 활동부터 K개의 활동까지 분석하여 세션이 완료될지 중간에 이탈할지 추론하는 실습입니다.

# 0) 패키지 설치
- 실습에 필요한 패키지를 설치합니다.

In [ ]:
!pip install imblearn

## 1) 환경 구성
- 실습을 원활하게 진행하기 위해 환경을 구성합니다.

In [ ]:
# === Cell 1. 환경 구성 ===
import os, re, json, inspect
from collections import Counter
from itertools import tee

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import dump

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix, average_precision_score
)
from sklearn.utils import check_random_state
from matplotlib.patches import Patch

_imb_ok = True
try:
    from imblearn.over_sampling import RandomOverSampler, SMOTE
    from imblearn.pipeline import Pipeline as ImbPipeline
except Exception as e:
    print("[warn] imbalanced-learn를 불러오지 못했습니다. 오버샘플링 비활성화:", e)
    _imb_ok = False

RANDOM_STATE = 42
rng = check_random_state(RANDOM_STATE)
TOPK_BIGRAMS = 50
K_LIST       = [3, 5, 7]
TERMINALS_FINAL = {"/logout", "/delete_user"}

OPER_POLICY = "balanced" 
T_OPER_DEFAULT = 0.730
OPER_T_OVERRIDE = {3: 0.700, 5: 0.700, 7: 0.620}

# 오버샘플링 옵션 - 한 쪽의 데이터가 너무 적을 시 사용
# 아래 K별 오버샘플링 옵션 사용 권장
OVERSAMPLING = {
    "enable": False,          # 전역 기본값 ( True = on / False = off )
    "method": "smote",        # "random" | "smote"
    "ratio": 0.5,             # 1.0=완전균형, 0.5=소수/다수=0.5
    "k_neighbors": 5,         # SMOTE용
    "random_state": RANDOM_STATE,
}

# K별 오버샘플링 override (전역 기본값을 덮어씀)
OVERSAMPLING_BY_K = {
    3: {"enable": False, "method": "smote", "ratio": 0.5, "k_neighbors": 5}, # K=3 
    5: {"enable": False, "method": "smote", "ratio": 0.5, "k_neighbors": 5}, # K=5
    7: {"enable": False, "method": "smote", "ratio": 0.5, "k_neighbors": 5}, # K=7
}

def _merge_oversampling_cfg(global_cfg: dict, per_k_cfg: dict) -> dict:
    cfg = dict(global_cfg)
    if per_k_cfg:
        cfg.update(per_k_cfg)
    return cfg

def _build_sampler(cfg: dict):
    """오버샘플러 생성(파이프라인 내부에서만 사용)."""
    if (not cfg.get("enable")) or (not _imb_ok):
        return None
    method = str(cfg.get("method", "random")).lower()
    ratio  = cfg.get("ratio", 1.0)
    rs     = cfg.get("random_state", RANDOM_STATE)
    if method == "random":
        return RandomOverSampler(sampling_strategy=ratio, random_state=rs)
    elif method == "smote":
        return SMOTE(sampling_strategy=ratio, random_state=rs, k_neighbors=int(cfg.get("k_neighbors", 5)))
    else:
        print(f"[warn] 알 수 없는 오버샘플링 방법: {method}. 비활성화합니다.")
        return None

# 입력/출력 경로
INPUT_PATH = "datasets/processed_user_behavior.sorted.csv"
OUT_DIR = "datasets/sessionDrop"
os.makedirs(OUT_DIR, exist_ok=True)

OUT_ALL = os.path.join(OUT_DIR, "sessionDrop.features.csv")
def OUT_PREFIXF_PATH(k:int) -> str:
    return os.path.join(OUT_DIR, f"sessionDrop.prefix{k}.features.filtered.csv")

MODEL_OUT_DIR = "models/sessionDrop"
os.makedirs(MODEL_OUT_DIR, exist_ok=True)
def MODEL_PATH_K(k:int) -> str:
    return os.path.join(MODEL_OUT_DIR, f"sessionDrop_model.k{k}.joblib")
def META_PATH_K(k:int) -> str:
    return os.path.join(MODEL_OUT_DIR, f"sessionDrop_model.k{k}.meta.json")

FEATURE_COLS = ["timestamp","current_state","page_depth",
                "delta_search_count","delta_cart_item_count","gap_sec"]

def make_param_dist(seed: int):
    r = check_random_state(seed)
    return {
        "n_estimators":      r.randint(600, 1601, size=20),
        "learning_rate":     r.choice([0.02, 0.03, 0.05, 0.07, 0.1], size=20),
        "max_depth":         r.choice([3,4,5,6,7], size=20),
        "min_child_weight":  r.choice([1,2,3,5], size=20),
        "subsample":         r.uniform(0.6, 1.0, size=20),
        "colsample_bytree":  r.uniform(0.6, 1.0, size=20),
        "reg_lambda":        r.choice([0.5,1.0,1.5,2.0], size=20),
        "reg_alpha":         r.choice([0.0,0.1,0.3,0.5], size=20),
    }

print("[Done]")

## 2) 원본 데이터 로드
- 원본 데이터를 로드하고 전체 세션의 개수를 집계합니다.

In [ ]:
# === Cell 2. 데이터 로드 ===
df = pd.read_csv(INPUT_PATH)

required = ["session_id","timestamp","current_state","next_state","page_depth","search_count","cart_item_count"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"필수 컬럼 누락: {missing}")

df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.dropna(subset=["session_id","timestamp"]).copy()
df["session_id"] = df["session_id"].astype(str)
df = df.sort_values(["session_id","timestamp"])

def norm_state(s: str) -> str:
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    s = re.sub(r"\?.*$","", s)
    if len(s)>1 and s.endswith("/"):
        s = s[:-1]
    return s

df["current_state"] = df["current_state"].map(norm_state)
df["next_state"]    = df["next_state"].map(norm_state)

print("Loaded rows:", len(df), "sessions:", df["session_id"].nunique())


## 3) 간격/증분 계산
- 세션 내 이벤트 간 시간차와 누적 지표의 증가분을 만들어 이후 피처 집계에 쓰기 좋게 정규화합니다.

In [ ]:
# === Cell 3. 간격/증분 생성 ===
df["gap_sec"] = (
    df.groupby("session_id")["timestamp"]
      .diff()
      .dt.total_seconds()
      .fillna(0)
      .clip(lower=0)
)

for col in ["page_depth","search_count","cart_item_count"]:
    dcol = f"delta_{col}"
    df[dcol] = df.groupby("session_id")[col].diff()
    df[dcol] = df[dcol].fillna(df[col]).clip(lower=0)

print("[Done]")

## 4) Targit Label 생성 & 누수 세션 식별
- 실제 결과인 Label을 생성하고, 모델 학습을 위해 첫 활동부터 K 활동 내에 실제 결과 값이 포함된 세션을 식별합니다.

In [ ]:
# === Cell 4. 세션 라벨 + prefix-K 및 누수 세션 식별 ===
is_final = df["current_state"].isin(TERMINALS_FINAL) | df["next_state"].isin(TERMINALS_FINAL)
labels = (
    df.assign(is_final=is_final)
      .groupby("session_id", sort=False)["is_final"]
      .any()
      .astype("int8")
      .reset_index(name="label")
)

prefix_by_k    = {}
term_sid_by_k  = {}

for K in K_LIST:
    prefix_k = (
        df.sort_values(["session_id","timestamp"])
          .groupby("session_id", group_keys=False)
          .head(K)
    )
    term_sid_k = prefix_k.loc[
        prefix_k["current_state"].isin(TERMINALS_FINAL) | prefix_k["next_state"].isin(TERMINALS_FINAL),
        "session_id"
    ].unique()
    prefix_by_k[K]   = prefix_k
    term_sid_by_k[K] = term_sid_k

print(f"[info] total sessions: {df['session_id'].nunique():,}")
for K in K_LIST:
    print(f"[info] K={K}: sessions reaching terminals in first {K} events (removed): {len(term_sid_by_k[K]):,}")

def bigrams(seq):
    a, b = tee(seq)
    next(b, None)
    for x, y in zip(a, b):
        yield (x, y)

bg_counter = Counter()
TERMINALS = TERMINALS_FINAL
for _, g in df.groupby("session_id", sort=False):
    seq = list(g["current_state"].fillna("").values)
    for bg in bigrams(seq):
        if (bg[0] in TERMINALS) or (bg[1] in TERMINALS):
            continue
        bg_counter[bg] += 1

top_bigrams = [bg for bg, _ in bg_counter.most_common(TOPK_BIGRAMS)]
bigram_to_idx = {bg:i for i,bg in enumerate(top_bigrams)}


## 5) Feature Engineering
- 모델을 학습하기 위해 데이터를 전처리합니다.

In [ ]:
# === Cell 5. 세션 피처 집계 함수 ===
def agg_features(g: pd.DataFrame) -> pd.Series:
    g = g.sort_values("timestamp")
    states = g["current_state"].fillna("").tolist()
    gaps   = g["gap_sec"].values

    feats = {}
    feats["n_events"] = len(g)
    feats["session_duration_sec"] = (
        (g["timestamp"].iloc[-1] - g["timestamp"].iloc[0]).total_seconds()
        if len(g)>1 else 0.0
    )
    feats["events_per_min"] = feats["n_events"] / max(feats["session_duration_sec"]/60.0, 1e-9)

    if len(gaps)>0:
        feats["gap_mean"] = float(np.mean(gaps))
        feats["gap_std"]  = float(np.std(gaps))
        feats["gap_p95"]  = float(np.quantile(gaps, 0.95))
    else:
        feats["gap_mean"]=feats["gap_std"]=feats["gap_p95"]=0.0

    feats["page_depth_max"]   = float(g["page_depth"].max())
    feats["page_depth_slope"] = float((g["page_depth"].iloc[-1] - g["page_depth"].iloc[0]) / max(len(g)-1,1))
    feats["depth_backtracks"] = int((g["page_depth"].diff()<0).sum())

    feats["search_delta_sum"] = float(g["delta_search_count"].sum())
    feats["search_rate"]      = feats["search_delta_sum"] / max(feats["n_events"],1)

    cart_delta = g["delta_cart_item_count"].clip(lower=0)
    feats["cart_adds"]        = float(cart_delta.sum())
    feats["cart_touch_count"] = int((cart_delta>0).sum())
    feats["cart_adds_per_event"] = feats["cart_adds"] / max(feats["n_events"],1)
    feats["search_to_cart_ratio"] = (feats["search_delta_sum"] + 1.0) / (feats["cart_adds"] + 1.0)

    first_ts = g["timestamp"].iloc[0]
    feats["first_event_hour"] = int(first_ts.hour)
    feats["is_weekend"]       = int(first_ts.weekday()>=5)

    def idx_of(pred_list, key):
        for i, s in enumerate(pred_list):
            if key in s: return i
        return None
    i_search = idx_of(states, "/search")
    i_cart   = idx_of(states, "/cart")
    feats["time_to_first_search"] = float((g["timestamp"].iloc[i_search] - first_ts).total_seconds()) if i_search is not None else -1.0
    feats["time_to_first_cart"]   = float((g["timestamp"].iloc[i_cart]   - first_ts).total_seconds()) if i_cart   is not None else -1.0

    lastk = states[-5:]
    joined_lastk = " ".join(lastk)
    feats["last5_has_search"]   = int("search" in joined_lastk)
    feats["last5_has_cart"]     = int("cart" in joined_lastk)
    feats["last5_has_checkout"] = int("checkout" in joined_lastk)

    bg_counts = Counter(list(bigrams(states)))
    for bg, idx in bigram_to_idx.items():
        feats[f"bg_{idx}"] = int(bg_counts.get(bg, 0))

    uniq = pd.Series(states).value_counts(normalize=True)
    feats["unique_states"] = int(uniq.size)
    feats["entropy_state"] = float(-(uniq*np.log(uniq+1e-12)).sum())

    st0 = states[0] if states else ""
    stL = states[-1] if states else ""
    for name, st in [("start", st0), ("last", stL)]:
        feats[f"{name}_is_root"]      = int(st == "/")
        feats[f"{name}_is_search"]    = int("/search" in st)
        feats[f"{name}_is_products"]  = int("/products" in st)
        feats[f"{name}_is_cart"]      = int("/cart" in st)
        feats[f"{name}_is_checkout"]  = int("/checkout" in st)

    all_states = " ".join(states)
    feats["hit_search"]   = int("/search" in all_states)
    feats["hit_products"] = int("/products" in all_states)
    feats["hit_cart"]     = int("/cart" in all_states)
    feats["hit_checkout"] = int("/checkout" in all_states)

    return pd.Series(feats)

print("[Done]")

## 6) 전처리 데이터 저장
- Feature Engineering 한 데이터를 K별로 저장합니다.

In [ ]:
# === Cell 6. 세션 피처 테이블 생성 & 저장 ===
X_all = (
    df.groupby("session_id", sort=False)[FEATURE_COLS]
      .apply(agg_features)
      .reset_index()
)
dataset_all = X_all.merge(labels, on="session_id", how="left")
dataset_all.to_csv(OUT_ALL, index=False)

datasets_by_k = {}
print("Saved CSVs:")
print(" -", OUT_ALL)

for K in K_LIST:
    X_prefix_k = (
        prefix_by_k[K].groupby("session_id", sort=False)[FEATURE_COLS]
                      .apply(agg_features)
                      .reset_index()
    )
    dataset_prefix_k = X_prefix_k.merge(labels, on="session_id", how="left")
    dataset_prefix_filt_k = dataset_prefix_k[~dataset_prefix_k["session_id"].isin(term_sid_by_k[K])].reset_index(drop=True)
    datasets_by_k[K] = dataset_prefix_filt_k
    out_path_k = OUT_PREFIXF_PATH(K)
    dataset_prefix_filt_k.to_csv(out_path_k, index=False)
    print(" -", out_path_k, "| shape:", tuple(dataset_prefix_filt_k.shape))
    
print("[Done]")

## 7) 학습용 / 검증용 데이터셋 분할
- 데이터를 학습용 / 검증용 데이터셋으로 분할합니다.

In [ ]:
# === Cell 7. 학습/검증 분할 (K별) ===
splits_by_k = {}
for K in K_LIST:
    dfp = datasets_by_k[K].copy()
    y = dfp["label"].astype(int)
    leak_cols = [c for c in dfp.columns if ("logout" in c.lower()) or ("delete" in c.lower())]
    X = dfp.drop(columns=["session_id","label"] + leak_cols, errors="ignore").fillna(0.0)

    X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.20, stratify=y, random_state=RANDOM_STATE)
    splits_by_k[K] = (X_tr, X_va, y_tr, y_va, X.columns.tolist())

    print(f"[K={K}] Shape: {X.shape}  PosRate: {float(y.mean()):.4f}")
    print(f"[K={K}] Train: {X_tr.shape}  Valid: {X_va.shape}")


## 8) 모델 선택
- 학습을 위한 모델을 선택합니다.

In [ ]:
# === Cell 8. 모델 선택 ===
use_xgb = False
try:
    from xgboost import XGBClassifier
    use_xgb = True
except Exception as e:
    print("[warn] xgboost 불가, LogisticRegression으로 폴백:", e)

best_by_k = {}             
use_pipeline = _imb_ok  

def _fixed_xgb(sampler_enabled: bool, y_tr):
    """튜닝 없이 사용할 고정 XGB 분류기 구성."""
    pos = int(y_tr.sum()); neg = int(len(y_tr) - pos)
    spw = 1.0 if sampler_enabled else (neg / max(pos, 1))
    return XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        n_estimators=800,
        learning_rate=0.05,
        max_depth=4,
        min_child_weight=2,
        subsample=0.85,
        colsample_bytree=0.80,
        reg_lambda=1.0,
        reg_alpha=0.1,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        scale_pos_weight=spw,
        verbosity=0,
    )

def _fixed_logreg(sampler_enabled: bool):
    """튜닝 없이 사용할 고정 로지스틱 회귀 구성."""
    from sklearn.linear_model import LogisticRegression
    return LogisticRegression(
        solver="saga",
        penalty="l2",
        C=1.0,
        max_iter=5000,
        class_weight=None if sampler_enabled else "balanced",
        n_jobs=-1,
    )

for K in K_LIST:
    X_tr, X_va, y_tr, y_va, _ = splits_by_k[K]

    cfg_k = _merge_oversampling_cfg(OVERSAMPLING, OVERSAMPLING_BY_K.get(K, {}))
    sampler = _build_sampler(cfg_k)
    sampler_enabled = sampler is not None

    if use_xgb:
        base_clf = _fixed_xgb(sampler_enabled, y_tr)
    else:
        base_clf = _fixed_logreg(sampler_enabled)

    if use_pipeline:
        steps = []
        if sampler_enabled:
            steps.append(("sampler", sampler))
        steps.append(("clf", base_clf))
        estimator = ImbPipeline(steps)
    else:
        estimator = base_clf

    clf_params = base_clf.get_params(deep=False)
    summary_keys = (
        ["n_estimators", "learning_rate", "max_depth", "min_child_weight",
         "subsample", "colsample_bytree", "reg_lambda", "reg_alpha"]
        if use_xgb else
        ["penalty", "C", "class_weight", "solver", "max_iter"]
    )
    used_params = {k: clf_params.get(k) for k in summary_keys if k in clf_params}

    best_by_k[K] = (estimator, used_params)
    print(f"[K={K}] Using {'XGBClassifier' if use_xgb else 'LogisticRegression'} Params:", used_params)


## 9) 모델 학습 & 평가
- 선택한 모델을 학습하고 검증용 데이터셋으로 평가합니다.

In [ ]:
# === Cell 9. 모델 학습 & 평가 ===

def safe_fit_xgb(model, Xtr, ytr, Xva, yva):
    """Pipeline 또는 단일 모델 모두 지원. XGBClassifier에 eval_set/early_stopping 전달."""
    is_pipe = hasattr(model, "named_steps") and ("clf" in getattr(model, "named_steps", {}))
    clf = model.named_steps["clf"] if is_pipe else model
    prefix = "clf__" if is_pipe else ""

    fit_kwargs = {}
    sig = inspect.signature(clf.fit)

    # eval_set
    if "eval_set" in sig.parameters:
        fit_kwargs[f"{prefix}eval_set"] = [(Xva, yva)]

    try:
        from xgboost.callback import EarlyStopping
        if "callbacks" in sig.parameters:
            fit_kwargs[f"{prefix}callbacks"] = [EarlyStopping(rounds=100, save_best=True)]
        elif "early_stopping_rounds" in sig.parameters:
            fit_kwargs[f"{prefix}early_stopping_rounds"] = 100
    except Exception:
        if "early_stopping_rounds" in sig.parameters:
            fit_kwargs[f"{prefix}early_stopping_rounds"] = 100

    if "verbose" in sig.parameters:
        fit_kwargs[f"{prefix}verbose"] = False

    model.fit(Xtr, ytr, **fit_kwargs)
    return model

SHOW_05 = False

trained_models_by_k = {}
eval_by_k = {}
T_OPER_BY_K = {}     # K별 운영 임계값 저장
summary_rows = []

SEP = "-" * 86 

for K in K_LIST:
    X_tr, X_va, y_tr, y_va, feat_names = splits_by_k[K]
    best_estimator_, best_params_ = best_by_k[K]

    final_model = best_estimator_

    if use_xgb:
        final_model = safe_fit_xgb(final_model, X_tr, y_tr, X_va, y_va)
    else:
        final_model.fit(X_tr, y_tr)


    p_va = final_model.predict_proba(X_va)[:, 1]

    if SHOW_05:
        yhat05 = (p_va >= 0.5).astype(int)
        print(f"[K={K}] [Valid @0.50] Acc={accuracy_score(y_va, yhat05):.4f} "
              f"Macro-F1={f1_score(y_va, yhat05, average='macro'):.4f} "
              f"PR-AUC={average_precision_score(y_va, p_va):.4f}")
        print(confusion_matrix(y_va, yhat05))
        print(classification_report(y_va, yhat05, digits=4))

    ths = np.linspace(0.01, 0.99, 197)
    def balanced_score(y_true, p, t):
        y_pred = (p >= t).astype(int)
        return 0.5*(accuracy_score(y_true, y_pred) + f1_score(y_true, y_pred, average="macro"))
    t_best, s_best = max(((t, balanced_score(y_va, p_va, t)) for t in ths), key=lambda x: x[1])

    if isinstance(OPER_T_OVERRIDE, dict) and (K in OPER_T_OVERRIDE):
        op_t = float(OPER_T_OVERRIDE[K])
    elif OPER_POLICY == "balanced":
        op_t = float(t_best)
    else:
        op_t = float(T_OPER_DEFAULT)
    T_OPER_BY_K[K] = op_t

    yhat_oper = (p_va >= op_t).astype(int)
    acc_oper = accuracy_score(y_va, yhat_oper)
    f1_oper  = f1_score(y_va, yhat_oper, average="macro")
    print(f"[K={K}] [Valid @t={op_t:.3f} (Oper/K-specific)] Acc={acc_oper:.4f} Macro-F1={f1_oper:.4f}")
    print(confusion_matrix(y_va, yhat_oper))
    print(classification_report(y_va, yhat_oper, digits=4))

    print(SEP)
    print()

    trained_models_by_k[K] = final_model
    eval_by_k[K] = {
        "Valid@Oper.Acc": float(acc_oper),
        "Valid@Oper.MacroF1": float(f1_oper),
        "Balanced.t": float(t_best),
        "Balanced.Score": float(s_best),
        "Oper.t": float(op_t),
        "features": feat_names,
    }
    summary_rows.append({
        "K": K,
        "Oper.t": float(op_t),
        "Valid@Oper.Acc": float(acc_oper),
        "Valid@Oper.MacroF1": float(f1_oper),
        "Balanced.t": float(t_best),
        "Balanced.Score": float(s_best),
    })

print("=== Summary (K별) ===")
for r in sorted(summary_rows, key=lambda x: x["K"]):
    print(f"K={r['K']} | Oper.t={r['Oper.t']:.3f} "
          f"| Acc@Oper={r['Valid@Oper.Acc']:.4f} "
          f"| MacroF1@Oper={r['Valid@Oper.MacroF1']:.4f} "
          f"| t_bal={r['Balanced.t']:.3f} | Score_bal={r['Balanced.Score']:.4f}")


## 10) 모델 저장
- K별로 모델을 저장합니다.

In [ ]:
# === Cell 10. 모델 & 메타 저장 (K별) ===

def to_builtin(v):
    if isinstance(v, (np.integer,)):  return int(v)
    if isinstance(v, (np.floating,)): return float(v)
    if isinstance(v, (np.bool_,)):    return bool(v)
    return v

print("Saving models & meta...")
for K in K_LIST:
    model = trained_models_by_k[K]
    best_params_ = best_by_k[K][1]
    feat_names   = eval_by_k[K]["features"]
    oper_t       = eval_by_k[K]["Oper.t"]   # K별 운영 임계값

    model_path = MODEL_PATH_K(K)
    dump(model, model_path)

    best_params_clean = {k: to_builtin(v) for k, v in (best_params_ or {}).items()}
    meta = {
        "variant": f"Baseline (prefix{K}; leakage-safe; no transform; no calibration)",
        "features": [str(c) for c in feat_names],
        "threshold": float(oper_t),
        "best_params": best_params_clean,
        "random_state": int(RANDOM_STATE),
        "preprocess": {
            "PREFIX_K": int(K),
            "TOPK_BIGRAMS": int(TOPK_BIGRAMS),
            "terminals": sorted(list(TERMINALS_FINAL)),
            "note": "prefix-K 내 terminal 등장 세션 제외, terminal 전이 제외 bigram 사전"
        },
        "oversampling": {
            "enabled": bool(OVERSAMPLING.get("enable", False) and _imb_ok),
            "method": OVERSAMPLING.get("method"),
            "ratio": float(OVERSAMPLING.get("ratio", 1.0)),
            "k_neighbors": int(OVERSAMPLING.get("k_neighbors", 5)),
        },
        "artifacts_csv": {
            "all": OUT_ALL,
            "prefix_filtered": OUT_PREFIXF_PATH(K)
        },
        "validation": {
            "Acc@Oper": eval_by_k[K]["Valid@Oper.Acc"],
            "MacroF1@Oper": eval_by_k[K]["Valid@Oper.MacroF1"],
            "Balanced_t": eval_by_k[K]["Balanced.t"],
            "Balanced_Score": eval_by_k[K]["Balanced.Score"],
        }
    }
    meta_path = META_PATH_K(K)
    with open(meta_path, "w") as f:
        json.dump(meta, f, indent=2)

    print(f" - Saved model: {model_path}")
    print(f" - Saved meta : {meta_path}")


## 11) 세션 이탈 추론 결과 값 시각화
- 검증한 모델을 평가한 결과 값을 그래프로 시각화합니다.

In [ ]:
# === Cell 11  ===

LABELS = [0, 1]
LABEL_NAMES = {0: "0 = Drop", 1: "1 = Complete"}

for K in K_LIST:
    X_tr, X_va, y_tr, y_va, _ = splits_by_k[K]
    model = trained_models_by_k[K]
    t = T_OPER_BY_K[K]

    p = model.predict_proba(X_va)[:, 1]
    yhat = (p >= t).astype(int)

    cm = confusion_matrix(y_va, yhat, labels=LABELS)
    cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
    acc = accuracy_score(y_va, yhat)
    macro_f1 = f1_score(y_va, yhat, average="macro")

    fig, ax = plt.subplots(figsize=(5, 4.5))

    for i in range(2):
        for j in range(2):
            correct = (i == j)
            ax.add_patch(plt.Rectangle(
                (j - 0.5, i - 0.5), 1, 1,
                facecolor=("green" if correct else "red"),
                edgecolor="black", alpha=0.35
            ))
            ax.text(
                j, i,
                f"{cm[i, j]}\n({cm_norm[i, j]:.2f})",
                ha="center", va="center", fontsize=12, fontweight="bold"
            )

    ax.set_xticks([0, 1]); ax.set_xticklabels([LABEL_NAMES[0], LABEL_NAMES[1]], rotation=15, ha="right")
    ax.set_yticks([0, 1]); ax.set_yticklabels([LABEL_NAMES[0], LABEL_NAMES[1]])
    ax.set_xlim(-0.5, 1.5); ax.set_ylim(1.5, -0.5)
    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
    ax.set_title(f"K={K} | @t={t:.3f} — Acc={acc:.4f}, Macro-F1={macro_f1:.4f}")

    legend_elems = [
        Patch(facecolor="green", edgecolor="black", label="Correct (TP/TN)"),
        Patch(facecolor="red", edgecolor="black", label="Incorrect (FP/FN)")
    ]
    ax.legend(handles=legend_elems, loc="upper right", frameon=True)

    plt.tight_layout()
    plt.show()